In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from my_functions import *

from scipy.optimize import curve_fit
from scipy.integrate import simpson

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))

In [ ]:
## Load QSO catalog
filename = ('/home/alberto/cosmos/JPAS_mocks_sep2021/'
    'JPAS_mocks_classification_01sep_model11/Fluxes/Qso_jpas_mock_flam_train.cat')

my_filter_order = np.arange(60)
my_filter_order[[-4, -3, -2, -1]] = np.array([1, 12, 28, 43])
my_filter_order[1:-4] += 1
my_filter_order[12:-4] += 1
my_filter_order[28:-4] += 1
my_filter_order[43:-4] += 1

pm_flx = pd.read_csv(
    filename, sep=' ', usecols=range(2, 2 + 60)
).to_numpy().T[my_filter_order]
pm_err = pd.read_csv(
    filename, sep=' ', usecols=range(2 + 60, 2 + 60 + 60)
).to_numpy().T[my_filter_order]
zspec = pd.read_csv(filename, sep=' ', usecols=[127]).to_numpy().reshape(-1, )

# Apply errors
np.random.seed(22)
pm_flx += pm_err * np.random.normal(size=pm_err.shape)

plate = pd.read_csv(filename, sep=' ', usecols=[122]).to_numpy().reshape(-1, )
mjd = pd.read_csv(filename, sep=' ', usecols=[123]).to_numpy().reshape(-1, )
fiber = pd.read_csv(filename, sep=' ', usecols=[124]).to_numpy().reshape(-1, )

In [ ]:
Lya_fts = pd.read_csv('csv/Lya_fts.csv')
EW_lya = np.abs(Lya_fts.LyaEW)
w_lya = 1215.67 # A
N_sources = pm_flx.shape[1]
mag = flux_to_mag(pm_flx, w_central.reshape(-1, 1))
mag[np.isnan(mag)] = 99.
zspec_dist = cosmo.luminosity_distance(zspec).to(u.cm).value
L = EW_lya * Lya_fts.LyaCont * 1e-17 * 4*np.pi * zspec_dist**2
L = np.log10(L)
L[np.isnan(L)] = -99

In [ ]:
ew0min = 30
ewobsmin = 100

# Lya search
cont_est_lya, cont_err_lya = estimate_continuum(pm_flx, pm_err)
line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, ew0min)
lya_lines, lya_cont_lines = identify_lines(line, pm_flx, pm_err, first=True)

# Other lines
cont_est_other, cont_err_other = estimate_continuum(pm_flx, pm_err, IGM_T_correct=False)
line_other = is_there_line(
    pm_flx, pm_err, cont_est_other, cont_err_other, ewobsmin, obs=True
)
other_lines = identify_lines(line_other, pm_flx, pm_err)

line_zero = is_there_line(
    pm_flx, pm_err, cont_est_other, cont_err_other, 0, obs=True
)

In [ ]:
z_Arr = np.zeros(N_sources)
z_Arr[np.where(np.array(lya_lines) != -1)] =\
    z_NB(np.array(lya_cont_lines)[np.where(np.array(lya_lines) != -1)])
nice_lya = nice_lya_select(lya_lines, other_lines, pm_flx, cont_est_other, z_Arr)

In [ ]:
np.random.seed(22)
selection = np.random.choice(np.where(nice_lya & (zspec > 2.5) & (mag[-2] < 22))[0], 15)
for src in selection:
    fig = plt.figure(figsize=(8, 6))
    ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src])

    ax.set_title('src{}'.format(src), fontsize=15)

    plt.show()

    print(plate[src], mjd[src], fiber[src])
    print('z_spec = {}'.format(zspec[src]))
    print('z_nb = {}'.format(z_Arr[src]))
    print('Lya EW0 = {}'.format(EW_lya[src]))
    print(src)

In [ ]:
# GAUSSIAN curve
def gauss(x, a, x0, sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

In [ ]:
tcurves = load_tcurves(load_filter_tags())

In [ ]:
for src in selection[:5]:
    line_start = np.where(line_zero[:, src])[0][0]
## TODO: START LOOKING AT LINES IN line_zero AT LYA POSITION
    idx = line_start + 1
    line_stop = line_start
    while True:
        if line[idx, src]:
            line_stop = idx
        else: break
        idx += 1 ; continue
    line_stop += 1 # So that the last index is included in everything

    nb_fit_range = slice(line_start, line_stop)

    s = (
        w_central[line_stop]
        - (w_central[line_start])
    ) * 0.5

    mu = (w_central[line_start] + w_central[line_stop]) / 2

    popt = ((2 * np.pi) ** -0.5 * s ** -1, mu, s)

    t_res = 10000 # Resolution for the t curve
    x_t = np.linspace(w_central[line_start], w_central[line_stop], t_res)
    composed_t = np.zeros(t_res)

    t_Arr = []
    for i in range(line_start, line_stop):
        t_Arr.append(np.interp(x_t, tcurves['w'][i], tcurves['t'][i]))
        composed_t += t_Arr[-1]
    t_Arr = np.array(t_Arr)

    lya_l = lya_lines[src]
    # Compute the area under the gaussian covolved with the filter T
    cvlvd_curve = gauss(x_t - cont_est_other[lya_l, src], *popt) * composed_t
    cvlvd_area = simpson(cvlvd_curve, x_t)

    nb_area = simpson(np.sum(t_Arr * pm_flx[nb_fit_range, src].reshape(-1,1), axis=0), x_t)

    correct_factor = nb_area / cvlvd_area

    fig = plt.figure(figsize=(10, 4))
    ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src])

    w_x = w_central[line_start - 2 : line_stop + 2]
    ax.plot(w_x, gauss(w_x, *popt) * correct_factor, linestyle='--', linewidth=3, c='black')

    plt.savefig('output/gauss_fit-src' + str(src) + '.pdf', bbox_inches='tight', dpi=600)
    plt.show()

In [ ]:
from astropy.table import Table

In [ ]:
spec = Table.read('../SDSS_Spectra/spec-4806-55688-0888.fits')

In [ ]:
fig = plt.figure(figsize=(10, 4))
ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src])

ax.plot(10 ** spec['loglam'], spec['flux'] * 1e-17, c='black', alpha=0.4, zorder=-99)
ax.plot(10 ** spec['loglam'], spec['model'] * 1e-17, c='red', zorder=-100)

w_x = w_central[line_start - 2 : line_stop + 2]
ax.plot(w_x, gauss(w_x, *popt) * correct_factor, linestyle='--', linewidth=3, c='black')

plt.savefig('output/gauss_fit-src' + str(src) + 's.pdf', bbox_inches='tight', dpi=600)
plt.show()